In [1]:
#import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from tiny.tfidf import *
from tiny.usage import *


@timed()
def gen_sub_by_para():
    args = locals()


    feature_label = get_stable_feature('rf01')

    train=feature_label[feature_label['sex'].notnull()]
    test =feature_label[feature_label['sex'].isnull()]

    X = train.drop(['sex', 'age', 'sex_age', 'device'], axis=1)



    Y = train['sex_age']
    Y_CAT = pd.Categorical(Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y_CAT.labels, test_size=0.3, random_state=666)

    # X_train.fillna(0, inplace=True)
    # X_test.fillna(0, inplace=True)

    # classifier = RandomForestClassifier(n_estimators=estimate,
    #                                     #criterion='entropy',
    #                                     verbose=1,
    #                                     n_jobs=-1,
    #                                     random_state=42)

    classifier = ExtraTreesClassifier(n_estimators=1000,
                                      max_depth=15,
                                      max_features=128,
                                      verbose=1,
                                      n_jobs=-1,
                                      random_state=42)


    print(f'Train begin#{args}')
    classifier.fit(X_train, y_train)
    print('Train End')


    pre_x=test.drop(['sex','age','sex_age','device'],axis=1)
    sub=pd.DataFrame(classifier.predict_proba(pre_x.values))


    sub.columns=Y_CAT.categories
    sub['DeviceID']=test['device'].values
    sub=sub[['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']]


    from sklearn.metrics import log_loss

    best = log_loss(y_test, classifier.predict_proba(X_test) )

    best = round(best, 4)

    #lgb.plot_importance(gbm, max_num_features=20)

    print(f'=============Final train feature({len(feature_label.columns)}):\n{list(feature_label.columns)} \n {len(feature_label.columns)}')

    file = f'./sub/baseline_rf_ex_{best}_{args}.csv'
    file = replace_invalid_filename_char(file)
    print(f'sub file save to {file}')
    sub = round(sub,10)
    sub.to_csv(file,index=False)
    
    print_imp_list(X_train, classifier)

if __name__ == '__main__':

    # for max_depth in range(15, 18, 1):
        #for n_estimators in range(6000, 60000, 2000):
            #for label in ['0902', 'rf01']:
                gen_sub_by_para()
    # gen_sub_by_para(True, 0.4)
    # for drop_long in np.arange(0.1, 1.1, 0.1):
    #     for drop_useless_pkg in [True, False]:
    #
    #         gen_sub_by_para(drop_useless_pkg, round(drop_long,1))


/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2018-09-23 19:34:33,677 util_log.py[33] INFO Begin to run gen_sub_by_para with:[], {}
2018-09-23 19:34:33,679 util_log.py[33] INFO Begin to run get_stable_feature with:['rf01'], {}
2018-09-23 19:34:33,682 util_cache_file.py[79] DEBUG fn:get_stable_feature, para:['rf01'], kw:{}
2018-09-23 19:34:33,683 util_cache_file.py[21] DEBUG try to read cache from file:./cache/get_stable_feature_['rf01']_{}.csv, type:csv
2018-09-23 19:34:36,171 util_cache_file.py[35] DEBUG Return 72727 resut from file cache:./cache/get_stable_feature_['rf01']_{}.csv
2018-0

Train begin#{}


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  4.4min finished


Train End


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    5.3s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    2.0s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    2.5s finished


=============Final train feature(135):
['device', '0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '1', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '2', '2.1', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '2.9', '3', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', 'package_0', 'package_1', 'package_2', 'package_3', 'package_4', 'package_5', 'package_6', 'duration_0', 'duration_1', 'duration_2', 'duration_3', 'duration_4', 'duration_5', 'duration_6', 'package_wk_0', 'package_wk_1', 'duration_wk_0', 'duration_wk_1', 'action_wk_0', 'action_wk_1', 'pkg_nunique', 'pkg_count', 'dur_sum', 'start_base_nunique', 'wk_compare_app_count', 'wk_comapre_dur', 'wk_compare_action_count', 'action_daily', 'dur_sum_daily', 'sum_pkg_nunique', 'sum_day_nunique', 'sum_span24_0_sum_p', 'sum_span24_1_sum_p', 'sum_span24_2_sum_p', 'sum_span24_3_sum_p', 'sum_span24_4_sum_p', 'sum_span24_5_

2018-09-23 19:39:08,199 util.py[393] INFO 001: 0.3                      , 0.026248948446602183, float64
2018-09-23 19:39:08,204 util.py[393] INFO 002: 1.3                      , 0.02098895648862325, float64
2018-09-23 19:39:08,206 util.py[393] INFO 003: 2.3                      , 0.019056909641512562, float64
2018-09-23 19:39:08,207 util.py[393] INFO 004: 2                        , 0.018046975909870693, float64
2018-09-23 19:39:08,211 util.py[393] INFO 005: 0.1                      , 0.01778197449579998, float64
2018-09-23 19:39:08,213 util.py[393] INFO 006: 0                        , 0.016791938523760222, float64
2018-09-23 19:39:08,215 util.py[393] INFO 007: 1.1                      , 0.014564987841287487, float64
2018-09-23 19:39:08,217 util.py[393] INFO 008: 1.4                      , 0.013870255063469608, float64
2018-09-23 19:39:08,219 util.py[393] INFO 009: 3                        , 0.01220083652814331, float64
2018-09-23 19:39:08,221 util.py[393] INFO 010: 4                   

2018-09-23 19:39:08,397 util.py[393] INFO 080: sum_span24_3_sum_p       , 0.006820533754736438, float64
2018-09-23 19:39:08,400 util.py[393] INFO 081: sum_span24_8_sum_p       , 0.006819476845717631, float64
2018-09-23 19:39:08,402 util.py[393] INFO 082: 4.6                      , 0.006779220448198873, float64
2018-09-23 19:39:08,405 util.py[393] INFO 083: sum_span24_6_sum_p       , 0.006772516982916383, float64
2018-09-23 19:39:08,406 util.py[393] INFO 084: sum_span24_9_sum_p       , 0.006769908299399999, float64
2018-09-23 19:39:08,409 util.py[393] INFO 085: 4.7                      , 0.006752045131384003, float64
2018-09-23 19:39:08,411 util.py[393] INFO 086: sum_span24_22_sum_p      , 0.0067489552795035965, float64
2018-09-23 19:39:08,413 util.py[393] INFO 087: 1.5                      , 0.006705364301694798, float64
2018-09-23 19:39:08,416 util.py[393] INFO 088: sum_span24_0_sum_p       , 0.006686015809773305, float64
2018-09-23 19:39:08,419 util.py[393] INFO 089: sum_span24_2_sum

In [1]:
#import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from tiny.tfidf import *
from tiny.usage import *

tmp = get_lda_app_and_usage(group_level='usage', drop=0, agg_col='package', agg_method='sum', n_topics=5) 

tmp.head()

/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2018-09-23 20:24:12,711 util_log.py[33] INFO Begin to run get_lda_app_and_usage with:[], {'group_level': 'usage', 'drop': 0, 'agg_col': 'package', 'agg_method': 'sum', 'n_topics': 5}
2018-09-23 20:24:12,714 util_cache_file.py[79] DEBUG fn:get_lda_app_and_usage, para:[], kw:{'group_level': 'usage', 'drop': 0, 'agg_col': 'package', 'agg_method': 'sum', 'n_topics': 5}
2018-09-23 20:24:12,716 util_log.py[33] INFO Begin to run get_cntTf with:['usage'], {'agg_col': 'package', 'agg_method': 'sum'}
2018-09-23 20:24:12,719 util_cache_file.py[79] DEBUG 

Try to lda for group_level#usage, agg_col#package, agg_method#sum
cntTf type:<class 'pandas.core.sparse.frame.SparseDataFrame'>
Convert df to csr_matrix
Lda analysis begin


/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Lda analysis end
Already calculate lda for <class 'type'> DF


2018-09-23 20:35:47,282 util_cache_file.py[47] DEBUG ====Write 72727 records to File#./cache/get_lda_app_and_usage_[]_{'group_level': 'usage', 'drop': 0, 'agg_col': 'package', 'agg_method': 'sum', 'n_topics': 5}.csv
2018-09-23 20:35:48,060 util_log.py[38] INFO cost: 695.35 sec: ==='get_lda_app_and_usage' end ([], {'group_level': 'usage', 'drop': 0, 'agg_col': 'package', 'agg_method': 'sum', 'n_topics': 5}) 


,device,usage_package_drop:0_0,usage_package_drop:0_1,usage_package_drop:0_2,usage_package_drop:0_3,usage_package_drop:0_4,usage_package_drop:0_app_length
0,00009270c4ec26e1d76f5d86847009c9,0.041125,0.041094,0.041094,0.049475,0.827213,3.0
1,000189ef5d5b951841d416a8c6c5b995,0.075211,0.056986,0.015174,0.106160,0.746468,19.0
2,00026d79a6f0955fc860947724e24765,0.198463,0.198463,0.198463,0.206148,0.198463,1.0
3,0002e3afb8146bc08e40575e45f0eca6,0.030678,0.030678,0.030678,0.030678,0.877290,3.0
4,0004709a296f9b925ae283efe2f043e7,0.034636,0.034957,0.813937,0.035118,0.081352,16.0
